In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
nomeFruta = input("Fruta? ")

Fruta? maca


In [3]:
with open("Calibracao_hsv_" + nomeFruta + ".txt") as hsv:
    
    parametros = hsv.readlines()
          
    hMin = int(parametros[0])
    sMin = int(parametros[1])
    vMin = int(parametros[2])
        
    hMax = int(parametros[3])
    sMax = int(parametros[4])
    vMax = int(parametros[5])

In [10]:
cap = cv2.VideoCapture(1)

while True:

    ret, frame = cap.read()
    hsv =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    imgPedunculo = np.copy(frame)

    lim_inf = np.array([hMin, sMin, vMin])
    lim_sup = np.array([hMax, sMax, vMax])

    color_mask = cv2.inRange(hsv, lim_inf, lim_sup)

    (couts,hir) = cv2.findContours(color_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    distanciaFocal = 1.185
    larguraFruta = 62
    distancia = 0.0
  
    for cout in couts:
        area = cv2.contourArea(cout)

        if (area > 800):

            x,y,w,h = cv2.boundingRect(cout)
            frame = cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

            #largura = (x + w)

            #distancia = (distanciaFocal * larguraFruta) / largura

            #distancia = round(distancia, 4)

            #cv2.putText(frame, str(distancia)+ "cm", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
            
            xt = x
            xb = x+w
            yt = y
            yb = y+h

            #Regiao de interesse

            Lmax = abs(xt - xb)
            Hmax = abs(yt - yb)

            RoiL = 0.5 * Lmax
            RoiH = 0.5 * Hmax

            #Esses coordenadas são as que vão delimitar a area onde a imagem vai ser cortada

            centro = (xb + xt) / 2

            centroCaixa = RoiL / 2

            x1 = centro - centroCaixa
            x2 = centro + centroCaixa

            y1 = abs(yt - 5)
            y2 = abs(RoiH - y1)
            
            centerPointX = (x1 + x2) / 2
            centerPointY = (y1 + y2) / 2
            
            centerPointX = int(centerPointX)
            centerPointY = int(centerPointY)
            
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)

            TopLeftY = y2
            TopLeftX = x1
            
            corte = imgPedunculo[y2:y1, x1:x2]

            #cv2.imwrite("teste.jpg", corte)
                                   
            #cv2.circle(frame, (centerPointX, centerPointY), 2,(255, 255, 0),  1)
            
            cv2.rectangle(frame, (x2, y2), (x1,y1), (255,255,0), 1)

    cv2.imshow('Identificacao por HSV', frame)
    cv2.imshow("ROI", corte)
   
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()    